In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Feb 20 18:32:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    54W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.2/660.2 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 72.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e3ad3c4275c1f04dda44730c7f7a400846ac82b2a431def8f2e9836710561b08
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
!pip3 install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=69874dfec8752e175a1f7f88cc0eb3c28bdd9a5881439d4628ed499d36e90fbb
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/32/HKT/global_config.py')
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/32/HKT')

In [ ]:
# import global_config

In [ ]:
visual_features_list=list(range(55,91))
acoustic_features_list=list(range(0,60))

ACOUSTIC_DIM = len(acoustic_features_list)
VISUAL_DIM = len(visual_features_list)
HCF_DIM=4
LANGUAGE_DIM=768

VISUAL_DIM_ALL = 91
ACOUSTIC_DIM_ALL = 81

H_MERGE_SENT = 768
DATASET_LOCATION = "./dataset/"
SEP_TOKEN_ID = 3

In [ ]:
import argparse
import csv
import logging
import os
import random
import pickle
import sys
# from global_config import *
import numpy as np
import wandb 

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from torch.nn import CrossEntropyLoss, L1Loss, BCEWithLogitsLoss
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef
from transformers import (
    AlbertConfig,
    AlbertTokenizer,
    AlbertForSequenceClassification,
    AutoTokenizer,
    BertForNextSentencePrediction,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)
# from models import *
from transformers.optimization import AdamW
import copy

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
import argparse
import csv
import logging
import os
import random
import pickle
import sys
# from global_config import *
import math
from torch.nn.utils.rnn import pad_sequence
import copy

import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (
    AlbertModel,
    AlbertPreTrainedModel,
    AlbertConfig,
    load_tf_weights_in_albert,
)
# from transformers.modeling_albert import AlbertEmbeddings, AlbertLayerGroup


In [ ]:
# parser = argparse.ArgumentParser()

In [ ]:

# parser.add_argument(
#     # "--model", type=str, choices=["HKT","language_only", "acoustic_only", "visual_only","hcf_only"], default="HKT",
#     "--model", type=str,  default="HKT",
# )

In [ ]:
# parser.add_argument("--dataset", type=str, choices=["sarcasm"], default="sarcasm")
# parser.add_argument("--batch_size", type=int, default=16)
# parser.add_argument("--max_seq_length", type=int, default=85)
# parser.add_argument("--n_layers", type=int, default=1)
# parser.add_argument("--n_heads", type=int, default=1)
# parser.add_argument("--cross_n_layers", type=int, default=1)
# parser.add_argument("--cross_n_heads", type=int, default=4)
# parser.add_argument("--fusion_dim", type=int, default=172)
# parser.add_argument("--dropout", type=float, default=0.2366)
# parser.add_argument("--epochs", type=int, default=20)

# parser.add_argument("--seed", type=int, default=100)

# parser.add_argument("--learning_rate", type=float, default=0.000005)
# parser.add_argument("--learning_rate_a", type=float, default=0.003)
# parser.add_argument("--learning_rate_h", type=float, default=0.0003)
# parser.add_argument("--learning_rate_v", type=float, default=0.003)
# parser.add_argument("--warmup_ratio", type=float, default=0.07178)
# parser.add_argument("--save_weight", type=str, choices=["True","False"], default="False")

In [ ]:
# def return_unk():
#     return 0





# args, unknown = parser.parse_known_args()
# # args = parser.parse_args()

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [ ]:
class Transformer(nn.Module):
    def __init__(self, d_model, num_layers=1, nhead=1, dropout=0.1, dim_feedforward=128, max_seq_length=5000):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.pos_encoder = nn.Embedding(max_seq_length, d_model)
        self.encoder = TransformerEncoder(TransformerLayer(d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout), num_layers=num_layers)
        self.decoder = nn.Linear(d_model, 1)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, input, attention_mask=None):
        seq_length = input.size()[1]
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input.device)
        positions_embedding = self.pos_encoder(position_ids).unsqueeze(0).expand(input.size()) # (seq_length, d_model) => (batch_size, seq_length, d_model)
        input = input + positions_embedding
        input = self.norm(input)
        hidden = self.encoder(input, attention_mask=attention_mask)
        out = self.decoder(hidden) # (batch_size, seq_len, hidden_dim)
        out = (out[:,0,:], out, hidden) # ([CLS] token embedding, full output, last hidden layer)
        return out



class TransformerLayer(nn.Module):
    def __init__(self, hidden_size, nhead=1, dim_feedforward=128, dropout=0.1):
        super(TransformerLayer, self).__init__()
        self.self_attention = Attention(hidden_size, nhead, dropout)
        self.fc = nn.Sequential(nn.Linear(hidden_size, dim_feedforward), nn.ReLU(), nn.Linear(dim_feedforward, hidden_size))
        self.norm1 = nn.LayerNorm(hidden_size)
        self.norm2 = nn.LayerNorm(hidden_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, attention_mask=None):
        src_1 = self.self_attention(src, src, attention_mask=attention_mask)
        src = src + self.dropout1(src_1)
        src = self.norm1(src)
        src_2 = self.fc(src)
        src = src + self.dropout2(src_2)
        src = self.norm2(src)

        return src


class TransformerEncoder(nn.Module):
    def __init__(self, layer, num_layers):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(layer, num_layers)
    def forward(self, src, attention_mask=None):
        for layer in self.layers:
            new_src = layer(src, attention_mask=attention_mask)
            src = src + new_src
        return src


class Attention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, attention_probs_dropout_prob, ctx_dim=None):
        super().__init__()
        if hidden_size % num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (hidden_size, num_attention_heads))
        self.num_attention_heads = num_attention_heads
        self.attention_head_size = int(hidden_size / num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        # visual_dim = 2048
        if ctx_dim is None:
            ctx_dim = hidden_size
        self.query = nn.Linear(hidden_size, self.all_head_size)
        self.key = nn.Linear(ctx_dim, self.all_head_size)
        self.value = nn.Linear(ctx_dim, self.all_head_size)

        self.dropout = nn.Dropout(attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, context, attention_mask=None):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(context)
        mixed_value_layer = self.value(context)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)

        # Apply the attention mask is 
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        return context_layer


class CrossAttentionLayer(nn.Module):
    def __init__(self, hidden_size, context_size, nhead=1, dropout=0.1):
        super(CrossAttentionLayer, self).__init__()
        self.src_cross_attention = Attention(hidden_size, nhead, dropout, ctx_dim=context_size)
        self.context_cross_attention = Attention(context_size, nhead, dropout, ctx_dim=hidden_size)
        self.self_attention = Attention(hidden_size + context_size, nhead, dropout)
        self.fc = nn.Sequential(nn.Linear(hidden_size + context_size, hidden_size + context_size), nn.ReLU())
        self.norm1 = nn.LayerNorm(hidden_size + context_size)
        self.norm2 = nn.LayerNorm(hidden_size + context_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, context, attention_mask=None):
        new_src = self.src_cross_attention(src, context, attention_mask=attention_mask)
        new_context = self.context_cross_attention(context, src, attention_mask=attention_mask)
        
        cross_src = torch.cat((new_src, new_context), dim=2)
        
        cross_src_1 = self.self_attention(cross_src, cross_src, attention_mask)
        cross_src = cross_src + self.dropout1(cross_src_1)
        cross_src = self.norm1(cross_src)

        cross_src_2 = self.fc(cross_src)
        cross_src = cross_src + self.dropout2(cross_src_2)
        cross_src = self.norm2(cross_src)

        return cross_src


        

class CrossAttentionEncoder(nn.Module):
    def __init__(self, layer, num_layers):
        super(CrossAttentionEncoder, self).__init__()
        self.layers = _get_clones(layer, num_layers)

    def forward(self, src, context, attention_mask=None):
        src_dim = src.size()[2]
        context_dim = context.size()[2]

        for layer in self.layers:
            output = layer(src, context, attention_mask=attention_mask)
            new_src = output[:,:,0:src_dim]
            new_context = output[:,:,src_dim:src_dim+context_dim]

            src = src + new_src
            context = context + new_context

        return output


#this version use multiple layer of cross attention
class HKTMultiLayerCrossAttn(nn.Module):
    
    def __init__(self, text_model, visual_model, acoustic_model,hcf_model, args, dropout=0.1,fusion_dim=128):
        
        super(HKTMultiLayerCrossAttn, self).__init__()
        self.newly_added_config=args
        self.text_model = text_model
        self.visual_model = visual_model
        self.acoustic_model = acoustic_model
        self.hcf_model = hcf_model
        
        L_AV_layer = CrossAttentionLayer((LANGUAGE_DIM+HCF_DIM), ACOUSTIC_DIM+VISUAL_DIM, nhead=args.cross_n_heads, dropout=args.dropout)
        self.L_AV = CrossAttentionEncoder(L_AV_layer, args.cross_n_layers)
        
        total_dim = 2 * (LANGUAGE_DIM+HCF_DIM+ ACOUSTIC_DIM + VISUAL_DIM )
        
        self.fc = nn.Sequential(nn.Linear(total_dim, args.fusion_dim), 
                                nn.ReLU(), 
                                nn.Dropout(args.dropout), 
                                nn.Linear(args.fusion_dim, 1))
        
    def get_params(self):
        
        acoustic_params=list(self.acoustic_model.named_parameters())
        visual_params=list(self.visual_model.named_parameters())
        hcf_params=list(self.hcf_model.named_parameters())
        
        other_params=list(self.text_model.named_parameters())+list(self.L_AV.named_parameters())+list(self.fc.named_parameters())
        
        return acoustic_params,visual_params,hcf_params,other_params
    

    def forward(self, input_ids, visual, acoustic,hcf, attention_mask=None, token_type_ids=None):
        
        (text_output, _) = self.text_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        (_, _, visual_output) = self.visual_model(visual)
        (_, _, acoustic_output) = self.acoustic_model(acoustic)
        (_, _, hcf_output) = self.hcf_model(hcf)
        
        
        text_hcf=torch.cat((text_output,hcf_output),dim=2)
        
        # attention mask conversion
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
        av_output=torch.cat((acoustic_output,visual_output),dim=2)
                
        noverbal_text=self.L_AV(text_hcf, av_output, attention_mask=extended_attention_mask)
        
        # Extract embeddings
        text_embedding = text_hcf[:,0,:] # [CLS] token
        visual_embedding = F.max_pool1d(visual_output.permute(0,2,1).contiguous(), visual_output.shape[1]).squeeze(-1)
        acoustic_embedding = F.max_pool1d(acoustic_output.permute(0,2,1).contiguous(),acoustic_output.shape[1]).squeeze(-1)
        L_AV_embedding = F.max_pool1d(noverbal_text.permute(0,2,1).contiguous(),noverbal_text.shape[1]).squeeze(-1)
        
        
        #print(weighted_vad_emb.shape)
        fusion = (text_embedding, visual_embedding, acoustic_embedding,L_AV_embedding)
        fused_hidden = torch.cat(fusion, dim=1)
        
        out = self.fc(fused_hidden)
        
        return (out, fused_hidden)






class HKT(nn.Module):
    def __init__(self, text_model, visual_model, acoustic_model,hcf_model, args, dropout=0.1,fusion_dim=128):
        super(HKT, self).__init__()
        
        self.newly_added_config=args
        self.text_model = text_model
        self.visual_model = visual_model
        self.acoustic_model = acoustic_model
        self.hcf_model = hcf_model
        
        self.L_AV = CrossAttentionLayer(LANGUAGE_DIM+HCF_DIM, ACOUSTIC_DIM+VISUAL_DIM, nhead=args.cross_n_heads, dropout=args.dropout)
        
        total_dim = 2 * (LANGUAGE_DIM+HCF_DIM + ACOUSTIC_DIM + VISUAL_DIM )
        
        self.fc = nn.Sequential(nn.Linear(total_dim, args.fusion_dim), 
                                nn.ReLU(), 
                                nn.Dropout(args.dropout), 
                                nn.Linear(args.fusion_dim, 1))
        
    def get_params(self):
        
        acoustic_params=list(self.acoustic_model.named_parameters())
        visual_params=list(self.visual_model.named_parameters())
        hcf_params=list(self.hcf_model.named_parameters())
        
        other_params=list(self.text_model.named_parameters())+list(self.L_AV.named_parameters())+list(self.fc.named_parameters())
        
        return acoustic_params,visual_params,hcf_params,other_params
    
    def forward(self, input_ids, visual, acoustic,hcf, attention_mask=None, token_type_ids=None):
        # print('input ids shape : ', input_ids.shape)
        # print('attention_mask shape : ', attention_mask.shape)
        (text_output, _) = self.text_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # text_output = self.text_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)['last_hidden_state']
        (_, _, visual_output) = self.visual_model(visual)
        (_, _, acoustic_output) = self.acoustic_model(acoustic)
        (_, _, hcf_output) = self.hcf_model(hcf)
        
        # print('self text model : \n')
        # print(self.text_model)
        # print()
        # print('text output type : ', type(text_output))
        # print('text_output shape : ', text_output.shape)
        # print('hcf_output shape : ', hcf_output.shape)
        text_hcf=torch.cat((text_output,hcf_output),dim=2)
        
        # attention mask conversion
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
        av_output=torch.cat((acoustic_output,visual_output),dim=2)
                
        noverbal_text=self.L_AV(text_hcf, av_output, attention_mask=extended_attention_mask)
        
        # Extract embeddings
        text_embedding = text_hcf[:,0,:] # [CLS] token
        visual_embedding = F.max_pool1d(visual_output.permute(0,2,1).contiguous(), visual_output.shape[1]).squeeze(-1)
        acoustic_embedding = F.max_pool1d(acoustic_output.permute(0,2,1).contiguous(),acoustic_output.shape[1]).squeeze(-1)
        L_AV_embedding = F.max_pool1d(noverbal_text.permute(0,2,1).contiguous(),noverbal_text.shape[1]).squeeze(-1)
        
        
        fusion = (text_embedding, visual_embedding, acoustic_embedding,L_AV_embedding)
        fused_hidden = torch.cat(fusion, dim=1)
        
        out = self.fc(fused_hidden)
        
        return (out, fused_hidden)




def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [ ]:
# class InputFeatures(object):
#     """A single set of features of data."""

#     def __init__(self, input_ids, input_mask, segment_ids, visual, acoustic,hcf,label_id):
#         self.input_ids = input_ids
#         self.input_mask = input_mask
#         self.segment_ids = segment_ids
#         self.visual = visual
#         self.acoustic = acoustic
#         self.hcf = hcf
#         self.label_id = label_id

# def _truncate_seq_pair(tokens_a, tokens_b, max_length):
#     """Truncates a sequence pair in place to the maximum length."""
#     pop_count = 0
#     while True:
#         total_length = len(tokens_a) + len(tokens_b)
#         if total_length <= max_length:
#             break
#         if len(tokens_a) == 0:
#             tokens_b.pop()
#         else:
#             pop_count += 1
#             tokens_a.pop(0)
#     return pop_count

# #albert tokenizer split words in to subwords. "_" marker helps to find thos sub words
# #our acoustic and visual features are aligned on word level. So we just create copy the same 
# #visual/acoustic vectors that belong to same word.
# def get_inversion(tokens, SPIECE_MARKER="▁"):
#     inversion_index = -1
#     inversions = []
#     for token in tokens:
#         if SPIECE_MARKER in token:
#             inversion_index += 1
#         inversions.append(inversion_index)
#     return inversions


# def convert_humor_to_features(examples, tokenizer, punchline_only=False):
#     features = []

#     for (ex_index, example) in enumerate(examples):
        
#         #p denotes punchline, c deontes context
#         #hid is the utterance unique id. these id's are provided by the authors of urfunny and mustard
#         #label is either 1/0 . 1=humor, 0=not humor
#         (
#             (p_words, p_visual, p_acoustic, p_hcf),
#             (c_words, c_visual, c_acoustic, c_hcf),
#             hid,
#             label
#         ) = example
                
#         text_a = ". ".join(c_words)
#         text_b = p_words + "."
#         tokens_a = tokenizer.tokenize(text_a)
#         tokens_b = tokenizer.tokenize(text_b)
        
#         inversions_a = get_inversion(tokens_a)
#         inversions_b = get_inversion(tokens_b)

#         pop_count = _truncate_seq_pair(tokens_a, tokens_b, args.max_seq_length - 3)

#         inversions_a = inversions_a[pop_count:]
#         inversions_b = inversions_b[: len(tokens_b)]

#         visual_a = []
#         acoustic_a = []
#         hcf_a=[]        
#         #our acoustic and visual features are aligned on word level. So we just 
#         #create copy of the same visual/acoustic vectors that belong to same word.
#         #because ber tokenizer split word into subwords
#         for inv_id in inversions_a:
#             visual_a.append(c_visual[inv_id, :])
#             acoustic_a.append(c_acoustic[inv_id, :])
#             hcf_a.append(c_hcf[inv_id, :])
            


#         visual_a = np.array(visual_a)
#         acoustic_a = np.array(acoustic_a)
#         hcf_a = np.array(hcf_a)
        
#         visual_b = []
#         acoustic_b = []
#         hcf_b = []
#         for inv_id in inversions_b:
#             visual_b.append(p_visual[inv_id, :])
#             acoustic_b.append(p_acoustic[inv_id, :])
#             hcf_b.append(p_hcf[inv_id, :])
        
#         visual_b = np.array(visual_b)
#         acoustic_b = np.array(acoustic_b)
#         hcf_b = np.array(hcf_b)
        
#         tokens = ["[CLS]"] + tokens_a + ["[SEP]"] + tokens_b + ["[SEP]"]

#         acoustic_zero = np.zeros((1, ACOUSTIC_DIM_ALL))
#         if len(tokens_a) == 0:
#             acoustic = np.concatenate(
#                 (acoustic_zero, acoustic_zero, acoustic_b, acoustic_zero)
#             )
#         else:
#             acoustic = np.concatenate(
#                 (acoustic_zero, acoustic_a, acoustic_zero, acoustic_b, acoustic_zero)
#             )

#         visual_zero = np.zeros((1, VISUAL_DIM_ALL))
#         if len(tokens_a) == 0:
#             visual = np.concatenate((visual_zero, visual_zero, visual_b, visual_zero))
#         else:
#             visual = np.concatenate(
#                 (visual_zero, visual_a, visual_zero, visual_b, visual_zero)
#             )
        
        
#         hcf_zero = np.zeros((1,4))
#         if len(tokens_a) == 0:
#             hcf = np.concatenate((hcf_zero, hcf_zero, hcf_b, hcf_zero))
#         else:
#             hcf = np.concatenate(
#                 (hcf_zero, hcf_a, hcf_zero, hcf_b, hcf_zero)
                
#             )
        
#         input_ids = tokenizer.convert_tokens_to_ids(tokens)

#         segment_ids = [0] * (len(tokens_a) + 2) + [1] * (len(tokens_b) + 1)
#         input_mask = [1] * len(input_ids)
            
#         acoustic_padding = np.zeros(
#             (args.max_seq_length - len(input_ids), acoustic.shape[1])
#         )
#         acoustic = np.concatenate((acoustic, acoustic_padding))
#         #original urfunny acoustic feature dimension is 81.
#         #we found many features are highly correllated. so we removed
#         #highly correlated feature to reduce dimension
#         acoustic=np.take(acoustic, acoustic_features_list,axis=1)
        
#         visual_padding = np.zeros(
#             (args.max_seq_length - len(input_ids), visual.shape[1])
#         )
#         visual = np.concatenate((visual, visual_padding))
#         #original urfunny visual feature dimension is more than 300.
#         #we only considred the action unit and face shape parameter features
#         visual = np.take(visual, visual_features_list,axis=1)
        
        
#         hcf_padding= np.zeros(
#             (args.max_seq_length - len(input_ids), hcf.shape[1])
#         )
        
#         hcf = np.concatenate((hcf, hcf_padding))
        
#         padding = [0] * (args.max_seq_length - len(input_ids))

#         input_ids += padding
#         input_mask += padding
#         segment_ids += padding

#         assert len(input_ids) == args.max_seq_length
#         assert len(input_mask) == args.max_seq_length
#         assert len(segment_ids) == args.max_seq_length
#         assert acoustic.shape[0] == args.max_seq_length
#         assert visual.shape[0] == args.max_seq_length
#         assert hcf.shape[0] == args.max_seq_length
        
#         label_id = float(label)
        
        
#         features.append(
#             InputFeatures(
#                 input_ids=input_ids,
#                 input_mask=input_mask,
#                 segment_ids=segment_ids,
#                 visual=visual,
#                 acoustic=acoustic,
#                 hcf=hcf,
#                 label_id=label_id,
#             )
#         )
            
#     return features



# def get_appropriate_dataset(data, tokenizer, parition):
    

#     features = convert_humor_to_features(data, tokenizer)
#     all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
#     all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
#     all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
#     all_visual = torch.tensor([f.visual for f in features], dtype=torch.float)
#     all_acoustic = torch.tensor([f.acoustic for f in features], dtype=torch.float)
#     hcf = torch.tensor([f.hcf for f in features], dtype=torch.float)
#     all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)
    

#     dataset = TensorDataset(
#         all_input_ids,
#         all_visual,
#         all_acoustic,
#         all_input_mask,
#         all_segment_ids,
#         hcf,
#         all_label_ids,
#     )
    
#     return dataset


# def set_up_data_loader():
#     if args.dataset=="humor":
#         data_file = "ur_funny.pkl"
#     elif args.dataset=="sarcasm":
#         # data_file = "mustard.pkl"
#         data_file = "/content/drive/MyDrive/Colab Notebooks/32/HKT/dataset/our_mustard_split_final.p"
#         # data_file = "/content/drive/MyDrive/Colab Notebooks/32/HKT/dataset/mustard.pkl"
        
#     with open(
#         # os.path.join(DATASET_LOCATION, data_file),
#         data_file,
#         "rb",
#     ) as handle:
#         all_data = pickle.load(handle)
        
#     train_data = all_data["train"]
#     dev_data = all_data["dev"]
#     test_data = all_data["test"]

#     # tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
#     tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

#     train_dataset = get_appropriate_dataset(train_data, tokenizer, "train")
    
#     dev_dataset = get_appropriate_dataset(dev_data, tokenizer, "dev")
#     print('Dev dataset : ', dev_dataset)
#     test_dataset = get_appropriate_dataset(test_data, tokenizer, "test")

#     train_dataloader = DataLoader(
#         train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=1
#     )

#     dev_dataloader = DataLoader(
#         dev_dataset, batch_size=args.batch_size, shuffle=True, num_workers=1
#     )

#     test_dataloader = DataLoader(
#         test_dataset, batch_size=args.batch_size, shuffle=True, num_workers=1
#     )
    
    
#     return train_dataloader, dev_dataloader, test_dataloader

# def train_epoch(model, train_dataloader, optimizer, scheduler, loss_fct):
#     model.train()
#     tr_loss = 0
#     nb_tr_examples, nb_tr_steps = 0, 0

#     for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

#         batch = tuple(t.to(DEVICE) for t in batch)
#         (
#             input_ids,
#             visual,
#             acoustic,
#             input_mask,
#             segment_ids,
#             hcf,
#             label_ids
#         ) = batch
        
#         visual = torch.squeeze(visual, 1)
#         acoustic = torch.squeeze(acoustic, 1)

#         if args.model == "language_only":
#             outputs = model(
#                 input_ids,
#                 token_type_ids=segment_ids,
#                 attention_mask=input_mask,
#                 labels=None,
#             )
#         elif args.model == "acoustic_only":
#             outputs = model(
#                 acoustic
#             )
#         elif args.model == "visual_only":
#             outputs = model(
#                 visual
#             )
#         elif args.model=="hcf_only":
#             outputs=model(hcf)
            
#         elif args.model=="HKT":
#             # print('\nhello worlld\n')
#             # print('input ids shape : ', input_ids.shape)
#             # print('visual shape : ', visual.shape)
#             # print('hcf shape : ', hcf.shape)
#             # print('segment ids shape : ', segment_ids.shape)
#             # print('attention mask shape : ', input_mask.shape)
#             outputs = model(input_ids, visual, acoustic,hcf, token_type_ids=segment_ids, attention_mask=input_mask,)
        
        
            
#         logits = outputs[0]
        
#         loss = loss_fct(logits.view(-1), label_ids.view(-1))

#         tr_loss += loss.item()
#         nb_tr_examples += input_ids.size(0)
#         nb_tr_steps += 1

#         loss.backward()
        
#         for o_i in range(len(optimizer)):
#             optimizer[o_i].step()
#             scheduler[o_i].step()
        
#         model.zero_grad()

#     return tr_loss/nb_tr_steps



# def eval_epoch(model, dev_dataloader, loss_fct):
    
#     model.eval()
#     dev_loss = 0
#     nb_dev_examples, nb_dev_steps = 0, 0
    
#     with torch.no_grad():
#         for step, batch in enumerate(tqdm(dev_dataloader, desc="Iteration")):
#             batch = tuple(t.to(DEVICE) for t in batch)
#             (
#                 input_ids,
#                 visual,
#                 acoustic,
#                 input_mask,
#                 segment_ids,
#                 hcf,
#                 label_ids
#             ) = batch
                    
#             visual = torch.squeeze(visual, 1)
#             acoustic = torch.squeeze(acoustic, 1)
    
#             if args.model == "language_only":
#                 outputs = model(
#                     input_ids,
#                     token_type_ids=segment_ids,
#                     attention_mask=input_mask,
#                     labels=None,
#                 )
#             elif args.model == "acoustic_only":
#                 outputs = model(
#                     acoustic
#                 )
#             elif args.model == "visual_only":
#                 outputs = model(
#                     visual
#                 )
#             elif args.model=="hcf_only":
#                 outputs=model(hcf)
                
#             elif args.model=="HKT":
#                 outputs = model(input_ids, visual, acoustic,hcf, token_type_ids=segment_ids, attention_mask=input_mask,)
            
            
#             logits = outputs[0]
#             loss = loss_fct(logits.view(-1), label_ids.view(-1))
    
#             dev_loss += loss.item()
#             nb_dev_examples += input_ids.size(0)
#             nb_dev_steps += 1

#     return dev_loss/nb_dev_steps

# def test_epoch(model, test_data_loader, loss_fct):
#     """ Epoch operation in evaluation phase """
#     model.eval()

#     eval_loss = 0.0
#     nb_eval_steps = 0
#     preds = []
#     all_labels = []

#     with torch.no_grad():
#         for step, batch in enumerate(tqdm(test_data_loader, desc="Iteration")):
            
#             batch = tuple(t.to(DEVICE) for t in batch)

#             (
#                 input_ids,
#                 visual,
#                 acoustic,
#                 input_mask,
#                 segment_ids,
#                 hcf,
#                 label_ids
#             ) = batch
                    
#             visual = torch.squeeze(visual, 1)
#             acoustic = torch.squeeze(acoustic, 1)
            
#             if args.model == "language_only":
#                 outputs = model(
#                     input_ids,
#                     token_type_ids=segment_ids,
#                     attention_mask=input_mask,
#                     labels=None,
#                 )
#             elif args.model == "acoustic_only":
#                 outputs = model(
#                     acoustic
#                 )
#             elif args.model == "visual_only":
#                 outputs = model(
#                     visual
#                 )
#             elif args.model=="hcf_only":
#                 outputs=model(hcf)
                
#             elif args.model=="HKT":
#                 # print('\nhello worlld\n')
#                 # print('input ids shape : ', input_ids.shape)
#                 # print('visual shape : ', visual.shape)
#                 # print('hcf shape : ', hcf.shape)
#                 # print('segment ids shape : ', segment_ids.shape)
#                 # print('attention mask shape : ', input_mask.shape)
#                 outputs = model(input_ids, visual, acoustic,hcf, token_type_ids=segment_ids, attention_mask=input_mask,)
            
            
#             logits = outputs[0]
            
            
#             tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

#             eval_loss += tmp_eval_loss.mean().item()
#             nb_eval_steps += 1
            
#             logits = torch.sigmoid(logits)
            
#             if len(preds) == 0:
#                 preds=logits.detach().cpu().numpy()
#                 all_labels=label_ids.detach().cpu().numpy()
#             else:
#                 preds= np.append(preds, logits.detach().cpu().numpy(), axis=0)
#                 all_labels = np.append(
#                     all_labels, label_ids.detach().cpu().numpy(), axis=0
#                 )
                
                
                
#         eval_loss = eval_loss / nb_eval_steps
#         preds = np.squeeze(preds)
#         all_labels = np.squeeze(all_labels)

#     return preds, all_labels, eval_loss



# def test_score_model(model, test_data_loader, loss_fct, exclude_zero=False):

#     predictions, y_test, test_loss = test_epoch(model, test_data_loader, loss_fct)
    
#     predictions = predictions.round()

#     f_score = f1_score(y_test, predictions, average="weighted")
#     accuracy = accuracy_score(y_test, predictions)

#     print("Accuracy:", accuracy,"F score:", f_score)
#     return accuracy, f_score, test_loss




# def train(
#     model,
#     train_dataloader,
#     dev_dataloader,
#     test_dataloader,
#     optimizer,
#     scheduler,
#     loss_fct,
# ):
       
#     best_valid_loss = 9e+9
#     # run_name = str(wandb.run.id)
#     run_name = 'colab'
#     valid_losses = []
    
#     n_epochs=args.epochs
        
    
#     for epoch_i in range(n_epochs):
        
#         train_loss = train_epoch(
#             model, train_dataloader, optimizer, scheduler, loss_fct
#         )
#         valid_loss = eval_epoch(model, dev_dataloader, loss_fct)

#         print('\n')
#         valid_losses.append(valid_loss)
#         print(
#             "\nepoch:{}, train_loss : {}, valid_loss : {}".format(
#                 epoch_i, train_loss, valid_loss
#             )
#         )
#         print('\n')

#         valid_accuracy, valid_f_score, valid_loss = test_score_model(
#             model, dev_dataloader, loss_fct
#         )

#         print('Valid accuracy : ', valid_accuracy)
#         print('Valid F score : ', valid_f_score)
#         print('\n')

#         test_accuracy, test_f_score, test_loss = test_score_model(
#             model, test_dataloader, loss_fct
#         )
        
#         print('Test accuracy : ', test_accuracy)
#         print('Test F score : ', test_f_score)    
#         if(valid_loss <= best_valid_loss):
#             best_valid_loss = valid_loss
#             best_valid_test_accuracy = test_accuracy
#             best_valid_test_fscore= test_f_score
#             print("\nBest model\n")
#             print("best  test accuracy : ", best_valid_test_accuracy)
#             print('best  test fscore : ', best_valid_test_fscore)
#             torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/32/HKT/saved_model/epoch_'+str(epoch_i)+'_f1_'+str(best_valid_test_fscore))
#             print("\n")
            
#             # if(args.save_weight == "True"):
#             #     print("Best model\n")
#             #     print("best valid test accuracy : ", best_valid_test_accuracy)
#             #     print('best valid test fscore : ', best_valid_test_fscore)
#                 # torch.save(model.state_dict(),'./best_weights/'+run_name+'.pt')
        


#         #we report test_accuracy of the best valid loss (best_valid_test_accuracy)
#         # wandb.log(
#         #     {
#         #         "train_loss": train_loss,
#         #         "valid_loss": valid_loss,
#         #         "test_loss": test_loss,
#         #         "best_valid_loss": best_valid_loss,
#         #         "best_valid_test_accuracy": best_valid_test_accuracy,
#         #         "best_valid_test_fscore":best_valid_test_fscore
#         #     }
#         # )
        



# def get_optimizer_scheduler(params,num_training_steps,learning_rate=1e-5):
    
#     no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {
#             "params": [
#                 p for n, p in params if not any(nd in n for nd in no_decay)
#             ],
#             "weight_decay": 0.01,
#         },
#         {
#             "params": [
#                 p for n, p in params if any(nd in n for nd in no_decay)
#             ],
#             "weight_decay": 0.0,
#         },
#     ]
    
#     optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer,
#         num_warmup_steps=int(num_training_steps * args.warmup_ratio),
#         num_training_steps=num_training_steps,
#     )
    
#     return optimizer,scheduler

# def prep_for_training(num_training_steps):
    
    
#     if args.model == "language_only":
#         model = AlbertForSequenceClassification.from_pretrained(
#             "albert-base-v2", num_labels=1
#         )
#     elif args.model == "acoustic_only":
#         model = Transformer(ACOUSTIC_DIM, num_layers=args.n_layers, nhead=args.n_heads, dim_feedforward=args.fc_dim)
        
#     elif args.model == "visual_only":
#         model = Transformer(VISUAL_DIM, num_layers=args.n_layers, nhead=args.n_heads, dim_feedforward=args.fc_dim)
        
#     elif args.model=="hcf_only":
#         model=Transformer(HCF_DIM, num_layers=args.n_layers, nhead=args.n_heads, dim_feedforward=args.fc_dim)
        
#     elif args.model == "HKT" :
#         #HKT model has 4 unimodal encoders. But the language one is ALBERT pretrained model. But other enocders are
#         #trained from scratch with low level features. We have found that many times most of the the gardients flows to albert encoders only as it
#         #already has rich contextual representation. So in the beginning the gradient flows ignores other encoders which are trained from low level features. 
#         # We found that if we intitalize the weights of the acoustic, visual and hcf encoders of HKT model from the best unimodal models that we already ran for ablation study then
#         #the model converege faster. Other wise it takes very long time to converge. 
#         if args.dataset=="humor":
#             visual_model = Transformer(VISUAL_DIM, num_layers=7, nhead=3, dim_feedforward= 128)
#             visual_model.load_state_dict(torch.load("./model_weights/init/humor/humorVisualTransformer.pt"))
#             acoustic_model = Transformer(ACOUSTIC_DIM, num_layers=8, nhead=3, dim_feedforward = 256)
#             acoustic_model.load_state_dict(torch.load("./model_weights/init/humor/humorAcousticTransformer.pt"))
#             hcf_model = Transformer(HCF_DIM, num_layers=3, nhead=2, dim_feedforward = 128)
#             hcf_model.load_state_dict(torch.load("./model_weights/init/humor/humorHCFTransformer.pt"))
            
#         elif args.dataset=="sarcasm":
#             visual_model = Transformer(VISUAL_DIM, num_layers=8, nhead=4, dim_feedforward=1024)
#             # visual_model.load_state_dict(torch.load("./model_weights/init/sarcasm/sarcasmVisualTransformer.pt"))
#             visual_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/init/sarcasm/sarcasmVisualTransformer.pt"))
#             acoustic_model = Transformer(ACOUSTIC_DIM, num_layers=1, nhead=3, dim_feedforward=512)
#             # acoustic_model.load_state_dict(torch.load("./model_weights/init/sarcasm/sarcasmAcousticTransformer.pt"))
#             acoustic_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/init/sarcasm/sarcasmAcousticTransformer.pt"))
#             hcf_model = Transformer(HCF_DIM, num_layers=8, nhead=4, dim_feedforward=128)
#             # hcf_model.load_state_dict(torch.load("./model_weights/init/sarcasm/sarcasmHCFTransformer.pt"))
#             hcf_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/init/sarcasm/sarcasmHCFTransformer.pt"))
        
#         text_model = AlbertModel.from_pretrained('albert-base-v2')
        
#         # print('Text model\n')
#         # print(text_model)
#         # print('\n')
#         num_parameters = sum(p.numel() for p in text_model.parameters())
#         print('text model Number of parameters : ', num_parameters/1e6)
#         model = HKT(text_model, visual_model, acoustic_model,hcf_model, args)

#     else:
#         raise ValueError("Requested model is not available")

#     model.to(DEVICE)
    
#     loss_fct = BCEWithLogitsLoss()
    

#     # Prepare optimizer
#     # used different learning rates for different componenets.
    
#     if args.model == "HKT" :
        
#         acoustic_params,visual_params,hcf_params,other_params = model.get_params()
#         optimizer_o,scheduler_o=get_optimizer_scheduler(other_params,num_training_steps,learning_rate=args.learning_rate)
#         optimizer_h,scheduler_h=get_optimizer_scheduler(hcf_params,num_training_steps,learning_rate=args.learning_rate_h)
#         optimizer_v,scheduler_v=get_optimizer_scheduler(visual_params,num_training_steps,learning_rate=args.learning_rate_v)
#         optimizer_a,scheduler_a=get_optimizer_scheduler(acoustic_params,num_training_steps,learning_rate=args.learning_rate_a)
        
#         optimizers=[optimizer_o,optimizer_h,optimizer_v,optimizer_a]
#         schedulers=[scheduler_o,scheduler_h,scheduler_v,scheduler_a]
        
#     else:
#         params = list(model.named_parameters())

#         optimizer_l, scheduler_l = get_optimizer_scheduler(
#             params, num_training_steps, learning_rate=args.learning_rate
#         )
        
#         optimizers=[optimizer_l]
#         schedulers=[scheduler_l]
        
        
#     return model, optimizers, schedulers,loss_fct




# def set_random_seed(seed):
#     """
#     This function controls the randomness by setting seed in all the libraries we will use.
#     """
#     random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
    
#     torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.enabled = False
#     torch.backends.cudnn.deterministic = True

    

# def main():
    
#     # wandb.init(project="HKT")
#     # wandb.config.update(args)
    
#     if(args.seed == -1):
#         seed = random.randint(0, 9999)
#         print("seed",seed)
#     else:
#         seed = args.seed
    
#     # wandb.config.update({"seed": seed}, allow_val_change=True)
    
#     set_random_seed(seed)
    
#     train_dataloader,dev_dataloader,test_dataloader=set_up_data_loader()
#     print("Dataset Loaded: ",args.dataset)
#     print('\n')
#     num_training_steps = len(train_dataloader) * args.epochs
    
#     model, optimizers, schedulers, loss_fct = prep_for_training(
#         num_training_steps
#     )
#     print("Model Loaded: ",args.model)

#     print('train loader length : ', len(train_dataloader.dataset))
#     print('dev loader length : ', len(dev_dataloader.dataset))
#     print('test loader length : ', len(test_dataloader.dataset))
#     train(
#         model,
#         train_dataloader,
#         dev_dataloader,
#         test_dataloader,
#         optimizers,
#         schedulers,
#         loss_fct,
#     )

#     # print("Full model : \n")
#     # print(model)
   
#     # z = torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/best/sarcasm/sarcasmHKT.pt")
#     # print(type(z))
#     # for name in z:
#     #   print(name)

#     # model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/best/sarcasm/sarcasmHKT.pt"))  
#     # model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/best/sarcasm/sarcasmAcousticTransformer.pt"))  
#     # print(z)
#     # test_accuracy, test_f_score, test_loss = test_score_model(
#     #         model, test_dataloader, loss_fct
#     #     )

#     # test_accuracy, test_f_score, test_loss = test_score_model(
#     #         model, train_dataloader, loss_fct
#     #     )
    
#     # print('Test accuracy : ', test_accuracy)
#     # print('Test F score : ', test_f_score)
    

# if __name__ == "__main__":
#     main()

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('albert-base-v2')
# model = AlbertModel.from_pretrained("albert-base-v2")


In [ ]:
# model

In [ ]:
# import argparse
# import csv
# import logging
# import os
# import random
# import pickle
# import sys
# from global_config import *
# import numpy as np

# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import precision_recall_fscore_support
# from sklearn.metrics import accuracy_score, f1_score

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
# from torch.utils.data.distributed import DistributedSampler
# from tqdm import tqdm, trange

# from torch.nn import CrossEntropyLoss, L1Loss, BCEWithLogitsLoss
# from scipy.stats import pearsonr, spearmanr
# from sklearn.metrics import matthews_corrcoef
# from transformers import (
#     AlbertConfig,
#     AlbertTokenizer,
#     AlbertForSequenceClassification,
#     BertForNextSentencePrediction,
#     BertTokenizer,
#     get_linear_schedule_with_warmup,
# )
# # from models import *
# from transformers.optimization import AdamW

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
def return_unk():
    return 0


parser.add_argument(
    # "--model", type=str, choices=["language_only", "acoustic_only", "visual_only","hcf_only","HKT"], default="HKT",
    "--model", type=str,  default="HKT"
)

_StoreAction(option_strings=['--model'], dest='model', nargs=None, const=None, default='HKT', type=<class 'str'>, choices=None, help=None, metavar=None)

In [ ]:


# # parser.add_argument("--dataset", type=str, choices=["humor", "sarcasm"], default="sarcasm")#humor=UR-FUNNY, sarcasm=MUsTARD
parser.add_argument("--dataset", type=str, choices=["sarcasm"], default="sarcasm")




_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='sarcasm', type=<class 'str'>, choices=['sarcasm'], help=None, metavar=None)

In [ ]:
parser.add_argument("--batch_size", type=int, default=16)
parser.add_argument("--max_seq_length", type=int, default=77)
parser.add_argument("--max_concept_length", type=int, default=5)
parser.add_argument("--n_layers", type=int, default=1)
parser.add_argument("--n_heads", type=int, default=1)
parser.add_argument("--cross_n_layers", type=int, default=1)
parser.add_argument("--cross_n_heads", type=int, default=2)
parser.add_argument("--fusion_dim", type=int, default=172)
parser.add_argument("--dropout", type=float, default=0.09379)
parser.add_argument("--seed", type=int, default=5149)



_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=5149, type=<class 'int'>, choices=None, help=None, metavar=None)

In [ ]:

args, unknown = parser.parse_known_args()

In [ ]:
global_train_data = []
global_dev_data = []
global_test_data = []

In [ ]:

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, visual, acoustic,hcf,label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.visual = visual
        self.acoustic = acoustic
        self.hcf = hcf
        self.label_id = label_id

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    pop_count = 0
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) == 0:
            tokens_b.pop()
        else:
            pop_count += 1
            tokens_a.pop(0)
    return pop_count


def get_inversion(tokens, SPIECE_MARKER="▁"):
    inversion_index = -1
    inversions = []
    for token in tokens:
        if SPIECE_MARKER in token:
            inversion_index += 1
        inversions.append(inversion_index)
    return inversions


def convert_humor_to_features(examples, tokenizer, punchline_only=False):
    features = []

    for (ex_index, example) in enumerate(examples):
        
        (
            (p_words, p_visual, p_acoustic, p_hcf),
            (c_words, c_visual, c_acoustic, c_hcf),
            hid,
            label
        ) = example
                
        text_a = ". ".join(c_words)
        text_b = p_words + "."
        tokens_a = tokenizer.tokenize(text_a)
        tokens_b = tokenizer.tokenize(text_b)
        
        inversions_a = get_inversion(tokens_a)
        inversions_b = get_inversion(tokens_b)

        pop_count = _truncate_seq_pair(tokens_a, tokens_b, args.max_seq_length - 3)

        inversions_a = inversions_a[pop_count:]
        inversions_b = inversions_b[: len(tokens_b)]

        visual_a = []
        acoustic_a = []
        hcf_a=[]        

        for inv_id in inversions_a:
            visual_a.append(c_visual[inv_id, :])
            acoustic_a.append(c_acoustic[inv_id, :])
            hcf_a.append(c_hcf[inv_id, :])
            


        visual_a = np.array(visual_a)
        acoustic_a = np.array(acoustic_a)
        hcf_a = np.array(hcf_a)
        
        visual_b = []
        acoustic_b = []
        hcf_b = []
        for inv_id in inversions_b:
            visual_b.append(p_visual[inv_id, :])
            acoustic_b.append(p_acoustic[inv_id, :])
            hcf_b.append(p_hcf[inv_id, :])
        
        visual_b = np.array(visual_b)
        acoustic_b = np.array(acoustic_b)
        hcf_b = np.array(hcf_b)
        
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"] + tokens_b + ["[SEP]"]

        acoustic_zero = np.zeros((1, ACOUSTIC_DIM_ALL))
        if len(tokens_a) == 0:
            acoustic = np.concatenate(
                (acoustic_zero, acoustic_zero, acoustic_b, acoustic_zero)
            )
        else:
            acoustic = np.concatenate(
                (acoustic_zero, acoustic_a, acoustic_zero, acoustic_b, acoustic_zero)
            )

        visual_zero = np.zeros((1, VISUAL_DIM_ALL))
        if len(tokens_a) == 0:
            visual = np.concatenate((visual_zero, visual_zero, visual_b, visual_zero))
        else:
            visual = np.concatenate(
                (visual_zero, visual_a, visual_zero, visual_b, visual_zero)
            )
        
        
        hcf_zero = np.zeros((1,4))
        if len(tokens_a) == 0:
            hcf = np.concatenate((hcf_zero, hcf_zero, hcf_b, hcf_zero))
        else:
            hcf = np.concatenate(
                (hcf_zero, hcf_a, hcf_zero, hcf_b, hcf_zero)
                
            )
        
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        segment_ids = [0] * (len(tokens_a) + 2) + [1] * (len(tokens_b) + 1)
        input_mask = [1] * len(input_ids)
            
        acoustic_padding = np.zeros(
            (args.max_seq_length - len(input_ids), acoustic.shape[1])
        )
        acoustic = np.concatenate((acoustic, acoustic_padding))
        acoustic=np.take(acoustic, acoustic_features_list,axis=1)
        
        visual_padding = np.zeros(
            (args.max_seq_length - len(input_ids), visual.shape[1])
        )
        visual = np.concatenate((visual, visual_padding))
        visual = np.take(visual, visual_features_list,axis=1)
        
        
        hcf_padding= np.zeros(
            (args.max_seq_length - len(input_ids), hcf.shape[1])
        )
        
        hcf = np.concatenate((hcf, hcf_padding))
        
        padding = [0] * (args.max_seq_length - len(input_ids))

        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == args.max_seq_length
        assert len(input_mask) == args.max_seq_length
        assert len(segment_ids) == args.max_seq_length
        assert acoustic.shape[0] == args.max_seq_length
        assert visual.shape[0] == args.max_seq_length
        assert hcf.shape[0] == args.max_seq_length
        
        label_id = float(label)
        
        
        features.append(
            InputFeatures(
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                visual=visual,
                acoustic=acoustic,
                hcf=hcf,
                label_id=label_id,
            )
        )
            
    return features



def get_appropriate_dataset(data, tokenizer, parition):
    

    features = convert_humor_to_features(data, tokenizer)
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_visual = torch.tensor([f.visual for f in features], dtype=torch.float)
    all_acoustic = torch.tensor([f.acoustic for f in features], dtype=torch.float)
    hcf = torch.tensor([f.hcf for f in features], dtype=torch.float)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)
    

    dataset = TensorDataset(
        all_input_ids,
        all_visual,
        all_acoustic,
        all_input_mask,
        all_segment_ids,
        hcf,
        all_label_ids,
    )
    
    return dataset


def set_up_data_loader():
    if args.dataset=="humor":
        data_file = "ur_funny.pkl"
    elif args.dataset=="sarcasm":
        # data_file = "/content/drive/MyDrive/Colab Notebooks/32/HKT/dataset/mustard.pkl"
        data_file = "/content/drive/MyDrive/Colab Notebooks/32/HKT/dataset/our_mustard_split_final.p"
        # data_file = "mustard.pkl"
        
    with open(
        # os.path.join(DATASET_LOCATION, args.dataset, data_file),
        data_file,
        "rb",
    ) as handle:
        all_data = pickle.load(handle)
    train_data = all_data["train"]
    dev_data = all_data["dev"]
    test_data = all_data["test"]

    print('Train data : ', len(train_data))
    print('Dev data : ', len(dev_data))
    print('Test data : ', len(test_data))

    tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

    train_dataset = get_appropriate_dataset(train_data, tokenizer, "train")
    dev_dataset = get_appropriate_dataset(dev_data, tokenizer, "dev")
    test_dataset = get_appropriate_dataset(test_data, tokenizer, "test")

    train_dataloader = DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=1
    )

    dev_dataloader = DataLoader(
        dev_dataset, batch_size=args.batch_size, shuffle=False, num_workers=1
    )

    test_dataloader = DataLoader(
        test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=1
    )
    
    return (train_dataloader, dev_dataloader, test_dataloader)



def get_model():
    
    if args.model == "HKT" :
        
        if args.dataset=="humor":
            visual_model = Transformer(VISUAL_DIM, num_layers=7, nhead=3, dim_feedforward= 128)
            acoustic_model = Transformer(ACOUSTIC_DIM, num_layers=8, nhead=3, dim_feedforward = 256)
            hcf_model = Transformer(HCF_DIM, num_layers=3, nhead=2, dim_feedforward = 128)
            text_model = AlbertModel.from_pretrained('albert-base-v2')
            model = HKT(text_model, visual_model, acoustic_model,hcf_model, args)
            model.load_state_dict(torch.load("./model_weights/best/humor/humorHKT.pt"))
        elif args.dataset=="sarcasm":
            visual_model = Transformer(VISUAL_DIM, num_layers=8, nhead=4, dim_feedforward=1024)
            acoustic_model = Transformer(ACOUSTIC_DIM, num_layers=1, nhead=3, dim_feedforward=512)
            hcf_model = Transformer(HCF_DIM, num_layers=8, nhead=4, dim_feedforward=128)    
            text_model = AlbertModel.from_pretrained("albert-base-v2")
            model = HKT(text_model, visual_model, acoustic_model, hcf_model, args)
            # model.load_state_dict(torch.load("./model_weights/best/sarcasm/sarcasmHKT.pt"))
            # model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/model_weights/best/sarcasm/sarcasmHKT.pt"))
            model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/32/HKT/saved_model/epoch_7_f1_0.6720807726075505"))
            
    
    
    model.to(DEVICE)
    
    return model


def test_epoch(model, data_loader, loss_fct):
    """ Epoch operation in evaluation phase """
    model.eval()

    eval_loss = 0.0
    nb_eval_steps = 0
    preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(
            data_loader, mininterval=2, desc="  - (Validation)   ", leave=False
        ):
            batch = tuple(t.to(DEVICE) for t in batch)

            (
                input_ids,
                visual,
                acoustic,
                input_mask,
                segment_ids,
                hcf,
                label_ids
            ) = batch
                    
            visual = torch.squeeze(visual, 1)
            acoustic = torch.squeeze(acoustic, 1)

            
            if args.model == "HKT":
                outputs = model(input_ids, visual, acoustic,hcf, token_type_ids=segment_ids, attention_mask=input_mask,)
            
            logits = outputs[0]
            
            
            tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            
            logits = torch.sigmoid(logits)
            
            
            if len(preds) == 0:
                preds=logits.detach().cpu().numpy()
                all_labels=label_ids.detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                all_labels = np.append(
                    all_labels, label_ids.detach().cpu().numpy(), axis=0
                )
            
        eval_loss = eval_loss / nb_eval_steps
        
        preds = np.squeeze(preds)
        all_labels = np.squeeze(all_labels)

    return preds, all_labels, eval_loss

def test_score_model(model, test_data_loader, loss_fct, exclude_zero=False):

    predictions, y_test, test_loss = test_epoch(model, test_data_loader, loss_fct)
    
    predictions = predictions.round()

    f_score = f1_score(y_test, predictions, average="weighted")
    accuracy = accuracy_score(y_test, predictions)

    print("Accuracy, F score", accuracy, f_score)
    return accuracy, f_score, y_test, predictions


def set_random_seed(seed):
    """
    This function controls the randomness by setting seed in all the libraries we will use.
    """
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def main():
   
   set_random_seed(args.seed)
   (
        train_data_loader,
        dev_data_loader,
        test_data_loader,
    ) = set_up_data_loader()
   
   model = get_model()
   print("loaded")
   loss_fct = BCEWithLogitsLoss()
   acc, f_score, test_gold, test_pred = test_score_model(model, test_data_loader, loss_fct)
   dev_acc, dev_f_score, dev_gold, dev_pred = test_score_model(model, dev_data_loader, loss_fct)
  #  acc, f_score = test_score_model(model, train_data_loader, loss_fct)
   print('test accuracy : ', acc)
   print('test f_score : ', f_score)

   print('dev accuracy : ', dev_acc)
   print('dev f score : ', dev_f_score)
   
   return test_gold, test_pred
if __name__ == "__main__":
    test_gold, test_pred = main()


Train data :  540
Dev data :  68
Test data :  67


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

<ipython-input-27-861261a25c63>:183: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  all_visual = torch.tensor([f.visual for f in features], dtype=torch.float)


Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

loaded


Accuracy, F score 0.6716417910447762 0.6720807726075505


Accuracy, F score 0.75 0.7501622323166774
test accuracy :  0.6716417910447762
test f_score :  0.6720807726075505
dev accuracy :  0.75
dev f score :  0.7501622323166774


In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/32/HKT/dataset/our_mustard_split_final.p", 'rb') as f:
  dataset = pickle.load(f)


In [ ]:
len(dataset)

3

In [ ]:
test_dataset = dataset['test']
len(test_dataset)

67

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(test_gold, test_pred)

array([[22,  8],
       [14, 23]])

In [ ]:
for j in range(len(test_pred)):
  if((test_pred[j]==0) and (test_gold[j]==1)):
    print(j, " : ", test_dataset[j][2])


7  :  2_15
9  :  2_550
10  :  1_3707
12  :  1_971
13  :  2_504
15  :  1_12202
21  :  1_8052
24  :  1_8136
31  :  2_573
32  :  1_4967
35  :  2_24
39  :  2_282
53  :  2_548
57  :  1_5571


In [ ]:
for j in range(len(test_pred)):
  if((test_pred[j]==1) and (test_gold[j]==0)):
    print(j, " : ", test_dataset[j][2])

11  :  1_10190
14  :  2_221
19  :  1_5786
34  :  1_11609
47  :  1_3348
50  :  2_125
51  :  2_8
63  :  2_467


In [ ]:
for j in range(len(test_pred)):
  if((test_pred[j]==0) and (test_gold[j]==0)):
    print(j, " : ", test_dataset[j][2])

1  :  2_445
3  :  2_180
4  :  2_162
5  :  1_2423
8  :  2_205
16  :  2_481
23  :  2_13
25  :  2_181
26  :  2_387
30  :  2_278
33  :  2_44
37  :  2_243
41  :  2_210
42  :  1_467
46  :  2_439
48  :  2_590
49  :  1_7442
56  :  2_134
58  :  2_594
60  :  2_353
61  :  2_426
65  :  2_253


In [ ]:
for j in range(len(test_pred)):
  if((test_pred[j]==1) and (test_gold[j]==1)):
    print(j, " : ", test_dataset[j][2])

0  :  1_6426
2  :  2_508
6  :  2_260
17  :  1_3064
18  :  1_8827
20  :  2_390
22  :  1_7357
27  :  2_430
28  :  2_140
29  :  1_1973
36  :  1_10748
38  :  1_7281
40  :  1_537
43  :  1_213
44  :  2_351
45  :  1_7047
52  :  1_6683
54  :  1_3545
55  :  2_141
59  :  2_440
62  :  2_491
64  :  2_182
66  :  1_105


In [ ]:
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')


In [ ]:
# encoded_input.keys()

In [ ]:
# output = model(**encoded_input)

In [ ]:
# output.keys()

In [ ]:
# a,b  = model(input_ids = encoded_input['input_ids'], attention_mask = encoded_input['attention_mask'], token_type_ids = encoded_input['token_type_ids'])

In [ ]:
# a

In [ ]:
# b